# Collectl Dsk Log Analysis

## Functionalities
- Plot disk I/O utilization graphs.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    collectl.tar.gz
  ...
  [node-n]/
    collectl.tar.gz
```
A tarball `collectl.tar.gz` contains log files. The log file extension identifies the type of resource monitored:
- `.cpu.gz`: CPU monitoring log file.
- `.numa.gz`: memory monitoring log file.
- `.dsk.gz`: disk I/O monitoring log file.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2022-01-15-18-52-05"
# Ramp up duration (in sec)
RAMP_UP_DURATION = 720
# Ramp down duration (in sec)
RAMP_DOWN_DURATION = 360

########## DISK I/O
# Analyzed metric (options: "name", "reads", "rmerge", "rkbytes", "waitr", "writes", "wmerge", "wkbytes", "waitw",
# "request", "quelen", "wait", "svctim", "util")
COLLECTL_DSK_METRIC = "wait"
# Filter disks
COLLECTL_DISKS = None

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from parsers.collectl_parser import CollectlParser
from utils.utils import *

## Log Parsing

In [ ]:
# Get experiment time range, excluding ramping periods
experiment_start_time = get_experiment_start_time(EXPERIMENT_DIRNAME)
start_time = experiment_start_time + np.timedelta64(RAMP_UP_DURATION, 's')
experiment_end_time = get_experiment_end_time(EXPERIMENT_DIRNAME)
end_time = experiment_end_time - np.timedelta64(RAMP_DOWN_DURATION, 's')

In [ ]:
# Build data frames
dsk = pd.concat([
    pd.DataFrame.from_dict(CollectlParser(logfile, "dsk", start_time, end_time).parse()).assign(node_name=node_name)
    for node_name, logfile in get_collectl_dsk_logfiles(EXPERIMENT_DIRNAME)
], ignore_index=True)

In [ ]:
# Filter data frames
dsk = dsk[(dsk["hw_metric"] == COLLECTL_DSK_METRIC)]
if COLLECTL_DISKS:
    dsk = dsk[(dsk["hw_no"].isin(COLLECTL_DISKS))]

In [ ]:
dsk["hw_no"].unique()

In [ ]:
# (Re) Build columns
dsk["timestamp"] = dsk.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
dsk["window"] = dsk.apply(lambda r: int(r["timestamp"]), axis=1)
dsk["value"] = dsk.apply(lambda r: float(r["value"]), axis=1)

In [ ]:
# (Re) Create index
dsk.set_index("timestamp", inplace=True)
dsk.sort_index(inplace=True)

In [ ]:
# Get values
node_names = get_node_names(EXPERIMENT_DIRNAME)

## Disk Monitoring

In [ ]:
# Plot disk utilization (1-sec granularity)
fig = plt.figure(figsize=(24, len(node_names) * 12))
for (i, node_name) in enumerate(node_names):
    df = dsk[(dsk["node_name"] == node_name)]
    df = df.groupby(["window"])["value"].mean()
    ax = fig.add_subplot(len(node_names), 1, i + 1)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="line", title="%s - Disk Utilization" % node_name, xlabel="Time (seconds)",
            ylabel="%s" % COLLECTL_DSK_METRIC, grid=True)

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot disk utilization (millisec granularity)
if MIN_TIME and MAX_TIME:
    fig = plt.figure(figsize=(24, len(node_names) * 12))
    for (i, node_name) in enumerate(node_names):
        df = dsk[(dsk["node_name"] == node_name)]
        df = df[(df.index >= MIN_TIME) & (df.index <= MAX_TIME)]
        df = df.groupby(["timestamp", "hw_no"])["value"].mean()
        df = df.unstack()
        ax = fig.add_subplot(len(node_names), 1, i + 1)
        ax.set_xlim((df.index.min(), df.index.max()))
        ax.grid(alpha=0.75)
        df.plot(ax=ax, kind="line", title="%s - Disk Utilization" % node_name, xlabel="Time (seconds)",
                ylabel="%s" % COLLECTL_DSK_METRIC, grid=True, legend=True)